# Image Classification

## Problem Description
The data consists of images pertaining to 10 categories. 

__The task is to train a convolutional neural net in keras for classifying these images.__

With the advent of networks like Alexnet, VGG, Inception etc., building an architecture is no more an issue for generic classification tasks. The purpose here is to get familiar with Convolutional Architectures, Keras API & hyper parameter tuning and validating the results.

## Takeaways
You should have thorough understanding of the following concepts. 

1. Loss/Cost functions

2. Deep Convolutional Nets

3. Transfer Learning

4. Image Augmentation

5. How to train CNNs with less data

6. Validating results


## Load required library

In [128]:
import os
import cv2
import matplotlib.pyplot as plt
import random 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import shutil
import math
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import MaxPooling2D, Convolution2D
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.models import model_from_json, load_model
from keras.callbacks import EarlyStopping


%matplotlib inline

## Define constants

In [129]:
img_rows=224
img_cols=224
num_channel=3

num_epoch=100
batch_size=32


## Load the dataset

In [130]:

from subprocess import check_output
print(check_output(["ls", "../input/10_categories/10_categories"]).decode("utf8"))




test
train



In [131]:
PATH = os.getcwd()
DATA_PATH = os.path.join(PATH, '../input/10_categories/10_categories/train')
data_dir_list = os.listdir(DATA_PATH)
print(data_dir_list)

TRN_AUGMENTED = os.path.join(PATH , 'Trn_Augmented_Images')
TST_AUGMENTED = os.path.join(PATH , 'Tst_Augmented_Images')

['Faces', 'watch', 'BACKGROUND_Google', 'airplanes', 'car_side', 'grand_piano', 'Motorbikes', 'bonsai', 'Leopards', 'Faces_easy']


### Generate Test data

### Using Image Data Generator & flow_from_directory : with data augmentation

### Using Image Data Generator & flow_from_directory : with-out data augmentation

In [132]:
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [133]:
#Takes the path to a directory & generates batches of augmented data.

#Train data generator
train_generator = data_gen.flow_from_directory(
        DATA_PATH,
        target_size=(img_rows, img_cols), 
        batch_size=batch_size,
        class_mode='categorical', #One of "categorical", "binary", "sparse", "input", or None. 
                                  #Default: "categorical". 
                                  #Determines the type of label arrays that are returned
        color_mode='rgb', 
        shuffle=True,  
        subset="training")

Found 2391 images belonging to 10 classes.


In [134]:
#Validation data generator

val_generator = data_gen.flow_from_directory(
        DATA_PATH,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb', 
        shuffle=True, 
        subset="validation")

Found 592 images belonging to 10 classes.


## Data Analysis

In [135]:
#Lables assigned to each class of images
print(train_generator.class_indices)
print(val_generator.class_indices)

{'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'airplanes': 5, 'bonsai': 6, 'car_side': 7, 'grand_piano': 8, 'watch': 9}
{'BACKGROUND_Google': 0, 'Faces': 1, 'Faces_easy': 2, 'Leopards': 3, 'Motorbikes': 4, 'airplanes': 5, 'bonsai': 6, 'car_side': 7, 'grand_piano': 8, 'watch': 9}


In [136]:
# Dist of training data across classes
print(np.unique(train_generator.classes, return_counts=True))

# Dist of training data across classes
print(np.unique(val_generator.classes, return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32), array([300, 279, 279, 128, 512, 512,  83,  80,  64, 154]))
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32), array([ 74,  69,  69,  32, 127, 128,  20,  19,  16,  38]))


__Observation :__

1. Dataset is not balanced.

2. It is a multi-class classification problem, we will go ahead with __Accuracy__.

## Building a custom CNN from scratch : Baseline

### Model Building

#### Using custom CNN

In [137]:
def hand_crafter_CNN(nb_classes, learn_rate, input_shape ):
    model = Sequential()
    
    model.add(Convolution2D(filters=32,kernel_size=(3, 3),padding='same',input_shape=input_shape,activation='relu'))
    model.add(Convolution2D(filters=32,kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    
    adam = Adam(lr=learn_rate, beta_1=.9, beta_2=.99)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=adam)
    
    return(model)


model = hand_crafter_CNN(nb_classes=train_generator.num_classes, learn_rate=0.001, input_shape=(img_rows,img_cols,num_channel))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 222, 222, 32)      9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 111, 111, 64)      18496     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 109, 109, 64)      36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 186624)            0         
__________

#### Understand model parameters

In [138]:
#Get all configurations of the model
model.get_config()

{'name': 'sequential_5',
 'layers': [{'class_name': 'Conv2D',
   'config': {'name': 'conv2d_17',
    'trainable': True,
    'batch_input_shape': (None, 224, 224, 3),
    'dtype': 'float32',
    'filters': 32,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'same',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Conv2D',
   'config': {'name': 'conv2d_18',
    'trainable': True,
    'filters': 32,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',


In [139]:
#List of layes in the model
model.layers

In [140]:
#List of all weights & there shape in the model for all layers
model.weights

[<tf.Variable 'conv2d_17/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_17/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_18/kernel:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_18/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'conv2d_19/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_19/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2d_20/kernel:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv2d_20/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'dense_9/kernel:0' shape=(186624, 512) dtype=float32_ref>,
 <tf.Variable 'dense_9/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'dense_10/kernel:0' shape=(512, 10) dtype=float32_ref>,
 <tf.Variable 'dense_10/bias:0' shape=(10,) dtype=float32_ref>]

In [141]:
# List of all weights & there shape in the model for layer 0
model.layers[0].weights

# There are two types of weights
# 1. kernel weights
# 2. bias weight

[<tf.Variable 'conv2d_17/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d_17/bias:0' shape=(32,) dtype=float32_ref>]

In [142]:
#Get all weight values for layet 0
model.layers[0].get_weights()

[array([[[[-3.35625932e-02, -6.94226995e-02, -1.57837570e-03,
           -9.95608121e-02,  7.43492842e-02,  7.17514455e-02,
           -1.55843794e-02, -4.09637243e-02,  1.08887911e-01,
           -6.66553676e-02, -1.13935351e-01, -2.62470096e-02,
            3.69209051e-03,  4.10173237e-02,  8.06227773e-02,
           -4.12195921e-03, -5.52114099e-02, -2.65748128e-02,
            1.13043740e-01,  4.30296957e-02, -8.52352455e-02,
            4.26192284e-02, -7.29476064e-02, -9.97439027e-04,
            1.17130950e-01, -1.25390664e-01, -6.89833537e-02,
           -8.04458559e-03,  9.55077261e-02, -5.86813614e-02,
           -1.33220121e-01, -9.43872184e-02],
          [ 5.17432392e-02,  6.34924173e-02, -1.01171210e-01,
            7.56030679e-02, -4.58096936e-02,  1.72732919e-02,
            7.47928768e-02, -7.64481574e-02,  1.67523772e-02,
           -2.74459049e-02,  8.01254511e-02,  9.84902233e-02,
           -1.12054147e-01,  3.49012613e-02,  8.43389183e-02,
           -1.34539604e-

In [143]:
#Get all kernal weight values for layet 0
model.layers[0].get_weights()[0]

array([[[[-3.35625932e-02, -6.94226995e-02, -1.57837570e-03,
          -9.95608121e-02,  7.43492842e-02,  7.17514455e-02,
          -1.55843794e-02, -4.09637243e-02,  1.08887911e-01,
          -6.66553676e-02, -1.13935351e-01, -2.62470096e-02,
           3.69209051e-03,  4.10173237e-02,  8.06227773e-02,
          -4.12195921e-03, -5.52114099e-02, -2.65748128e-02,
           1.13043740e-01,  4.30296957e-02, -8.52352455e-02,
           4.26192284e-02, -7.29476064e-02, -9.97439027e-04,
           1.17130950e-01, -1.25390664e-01, -6.89833537e-02,
          -8.04458559e-03,  9.55077261e-02, -5.86813614e-02,
          -1.33220121e-01, -9.43872184e-02],
         [ 5.17432392e-02,  6.34924173e-02, -1.01171210e-01,
           7.56030679e-02, -4.58096936e-02,  1.72732919e-02,
           7.47928768e-02, -7.64481574e-02,  1.67523772e-02,
          -2.74459049e-02,  8.01254511e-02,  9.84902233e-02,
          -1.12054147e-01,  3.49012613e-02,  8.43389183e-02,
          -1.34539604e-01, -9.04875919e-

In [144]:
#Get all bias weight values for layet 0
model.layers[0].get_weights()[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

### Training the model

## Building a CNN using transfer Learning

### Define model

In [145]:
#Define input layer
image_input = Input(shape=(img_rows, img_cols, num_channel))

#Import VGG16 trained weights
model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

#### Take the trained weights only from last layer


#### Train the model

#### Take the trained weights only from last 3 layer

In [146]:
num_classes = train_generator.num_classes
last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)


#Define the new altered model
# we have replaced the last layer with our output layer
cust_vgg_model2 = Model(image_input, out)
cust_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

#### Train the model

In [147]:
for layer in cust_vgg_model2.layers[:-3]:
    layer.trainable = False

In [148]:
train_generator_num_samples = len(train_generator.filenames)
val_generator_num_samples = len(val_generator.filenames)

adam = Adam(lr=.001, beta_1=.9, beta_2=.99)
cust_vgg_model2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
cust_vgg_model2.fit_generator(train_generator, epochs=num_epoch, verbose=1, validation_data=val_generator, 
                              steps_per_epoch=math.ceil(train_generator_num_samples / batch_size), validation_steps=math.ceil(val_generator_num_samples / batch_size),
                             callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])


Epoch 1/100
75/75 [==============================] - 12s 158ms/step - loss: 0.4488 - acc: 0.8818 - val_loss: 0.0441 - val_acc: 0.9865
Epoch 2/100
75/75 [==============================] - 9s 121ms/step - loss: 0.0365 - acc: 0.9917 - val_loss: 0.0585 - val_acc: 0.9865


### Save the weights

In [149]:
# Serialize model to JSON
model_json = cust_vgg_model2.to_json()
with open("./model_image_1.json", "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
cust_vgg_model2.save_weights("./model_image_1.h5")


In [150]:
from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

__notebook_source__.ipynb
model_image_1.h5
model_image_1.json



### Load the weights

In [151]:
# Load json and create model
json_file = open('./model_image_1.json', 'r')
loaded_model_json_1 = json_file.read()
json_file.close()
loaded_model_json_1 = model_from_json(loaded_model_json_1)

# Load weights into new model
loaded_model_json_1.load_weights("./model_image_1.h5")

### Model evaluation on test data

In [152]:
#Test data, only original data NO augmentation
test_data_gen = ImageDataGenerator(rescale=1./255) 


#Validation data generator
TEST_DATA_PATH = "../input/10_categories/10_categories/test/"

test_generator = test_data_gen.flow_from_directory(
        TEST_DATA_PATH,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb', 
        shuffle=False)

test_generator_num_samples = len(test_generator.filenames)

Found 741 images belonging to 10 classes.


In [153]:
fd_model_evaluate = cust_vgg_model2.evaluate_generator(test_generator, verbose=1, steps=math.ceil(test_generator_num_samples / batch_size))
print("Loss: ", fd_model_evaluate[0], "Accuracy: ", fd_model_evaluate[1])

24/24 [==============================] - 3s 112ms/step
Loss:  0.07477175302985131 Accuracy:  0.9730094468545335


In [154]:
Y_pred = loaded_model_json_1.predict_generator(test_generator, verbose=1, steps=math.ceil(test_generator_num_samples / batch_size))
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)

24/24 [==============================] - 3s 117ms/step
[0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 0 5 5
 0 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 

## ToDo

1. Is balanced dataset?
2. What is the correct evaluation matrix?
3. Is data augmentation required?
